In [ ]:
# download nltk stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Install a particular version of `google-cloud-storage` because (oddly enough) 
# the  version on Colab and GCP is old. A dependency error below is okay.
!pip install -q google-cloud-storage==1.43.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 KB 3.8 MB/s eta 0:00:00


In [ ]:
# authenticate below for Google Storage access as needed
from google.colab import auth
auth.authenticate_user()

MessageError: ignored

In [ ]:
# install ngrok to emulate public IP / address
!wget -N https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip

In [ ]:
# TODO: sign up for an ngrok account
# then put your ngrok token below, uncomment, and execute
!./ngrok authtoken 22KeqcMQYpd2iRnsaZZOzlyaNw0_5NAk69pXRUMvfrf1oCNpU

In [ ]:
# install a ngrok python package and a version of flask that works with it in 
# colab
!pip -q install flask-ngrok
!pip -q install flask==0.12.2
# !pip -q install flask_restful

# Run the app

In [ ]:
# you need to upload your implementation of search_app.py
import search_frontend as se

In [ ]:
# uncomment the code below and execute to reload the module when you make 
# changes to search_frontend.py (after you upload again).
# import importlib
# importlib.reload(se)

In [ ]:
from flask_ngrok import run_with_ngrok
run_with_ngrok(se.app) 
se.app.run()

# Testing your app

Once your app is running you can query it. You can simply do that by navigating to the URL that ngrok gave you above or through code in a different python session. For example, once the frontend app is running, you can navigate to:
http://YOUR_SERVER_DOMAIN/search?query=hello+world where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io, which is printed above in Colab or that is your external IP on GCP.

The code below shows how to issue a query from python. This is also how our testing code will issue queries to your search engine, so make sure to test your search engine this way after you deploy it to GCP and before submission. Command line instructions for deploying your search engine to GCP are available at `run_frontend_in_gcp.sh`. Note that we will not only issue training queries to your search engine, but also test queries, i.e. queries that you've never seen before.

In [1]:
import json

with open('queries_train.json', 'rt') as f:
  queries = json.load(f)

In [2]:
def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):        
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)            
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

In [3]:
import requests
from time import time
# url = 'http://35.232.59.3:8080'
# place the domain you got from ngrok or GCP IP below. 
url = 'http://35.192.150.237:8080'
total_prs=0
qs_res = []
for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
    res = requests.get(url + '/search', {'query': q}, timeout=35)
    duration = time() - t_start
    if res.status_code == 200:
      pred_wids, _ = zip(*res.json())
      ap = average_precision(true_wids, pred_wids)
      total_prs+=ap
  except:
    pass
  
  qs_res.append((q, duration, ap))

In [4]:
print(qs_res)
print(total_prs/len(qs_res))
sum=0
for x in qs_res:
  sum+=x[1]
print(sum/len(qs_res))

[('best marvel movie', 4.223106861114502, 0.628), ('How do kids come to world?', 6.1919591426849365, 0.067), ('Information retrieval', 1.8919892311096191, 0.491), ('LinkedIn', 0.6271605491638184, 1.0), ('How to make coffee?', 2.4467480182647705, 0.499), ('Ritalin', 0.6102287769317627, 0.806), ('How to make wine at home?', 4.9940361976623535, 0.394), ('Most expensive city in the world', 8.653846740722656, 1.0), ('India', 1.541567087173462, 0.742), ('how to make money fast?', 3.4416253566741943, 1.0), ('Netflix', 0.7129387855529785, 1.0), ('Apple computer', 1.5050387382507324, 0.699), ('The Simpsons', 0.6626126766204834, 0.371), ('World cup', 5.923502445220947, 0.643), ('How to lose weight?', 1.353252649307251, 0.21), ('Java', 0.6943082809448242, 0.703), ('Air Jordan', 2.302251100540161, 0.226), ('how to deal with depression?', 1.619192361831665, 0.31), ('How do you make gold', 3.193948268890381, 0.46), ('Marijuana', 0.6596946716308594, 0.69), ('How to make hummus', 2.1200859546661377, 0